In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (classification_report, confusion_matrix, roc_auc_score, 
                             accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef)
from imblearn.over_sampling import SMOTE
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.initializers import HeNormal, GlorotUniform
import seaborn as sns

# Load data from CSV file
file_path = "Data.csv"
data = pd.read_csv(file_path)

# Assuming the last column is the target variable and all other columns are features
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Handle class imbalance using SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_scaled, y)

# Split the resampled data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Build the DNN model
initializer = HeNormal()
model = Sequential([
    Dense(256, activation='relu', kernel_initializer=initializer, input_shape=(X_train.shape[1],)),
    Dropout(0.4),
    Dense(128, activation='relu', kernel_initializer=initializer),
    Dropout(0.4),
    Dense(64, activation='relu', kernel_initializer=initializer),
    Dropout(0.3),
    Dense(32, activation='relu', kernel_initializer=initializer),
    Dropout(0.3),
    Dense(1, activation='softmax', kernel_initializer=GlorotUniform())  # Output layer with a single neuron for multiclass
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

# Generate predictions
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

# Metrics calculation
precision = precision_score(y_test, y_pred_classes, average='weighted')
recall = recall_score(y_test, y_pred_classes, average='weighted')
f1 = f1_score(y_test, y_pred_classes, average='weighted')
mcc = matthews_corrcoef(y_test, y_pred_classes)
auc = roc_auc_score(y_test, y_pred, average='weighted', multi_class='ovr')

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"MCC: {mcc:.4f}")
print(f"AUC: {auc:.4f}")

# Classification report
print(classification_report(y_test, y_pred_classes))

# Confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred_classes)
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

# Visualization of training history
fig, axes = plt.subplots(2, 5, figsize=(20, 10))
axes = axes.ravel()

# Accuracy plot
axes[0].plot(history.history['accuracy'], label='Training Accuracy')
axes[0].plot(history.history['val_accuracy'], label='Validation Accuracy')
axes[0].set_title('Model Accuracy')
axes[0].set_xlabel('Epochs')
axes[0].set_ylabel('Accuracy')
axes[0].legend()

# Loss plot
axes[1].plot(history.history['loss'], label='Training Loss')
axes[1].plot(history.history['val_loss'], label='Validation Loss')
axes[1].set_title('Model Loss')
axes[1].set_xlabel('Epochs')
axes[1].set_ylabel('Loss')
axes[1].legend()

# Predictions vs Actual (Sampled)
axes[2].scatter(range(len(y_test[:50])), y_test[:50], label='Actual', alpha=0.7)
axes[2].scatter(range(len(y_test[:50])), y_pred_classes[:50], label='Predicted', alpha=0.7)
axes[2].set_title('Predicted vs Actual (Sample)')
axes[2].legend()

# Histogram of predicted classes
axes[3].hist(y_pred_classes, bins=np.arange(0, len(np.unique(y)) + 1) - 0.5, alpha=0.7, color='blue')
axes[3].set_title('Histogram of Predicted Classes')

# Placeholder plots for remaining
for i in range(4, 10):
    axes[i].text(0.5, 0.5, f'Plot {i+1}', fontsize=15, ha='center')
    axes[i].set_axis_off()

plt.tight_layout()
plt.show()
